# Языковое моделирование

In [31]:
# Импорт библиотек

import tensorflow as tf

import numpy as np
import os
import time

In [33]:
# Загрузка текста

file = 'evgenyi_onegin.txt'

text = open(file, 'rb').read().decode(encoding='utf-8')

print('Длина текста: {} символов'.format(len(text)))

Длина текста: 286984 символов


In [34]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [35]:
# Увеличение количества текста

text = text + text

In [36]:
# The unique characters in the file

vocab = sorted(set(text))
print('Уникальных символов: {}'.format(len(vocab)))

Уникальных символов: 131


In [37]:
# Creating a mapping from unique characters to indices

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

len(text_as_int), len(text)

(573968, 573968)

In [38]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [39]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [40]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [41]:
# Пример вводных данных и таргета

for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


Создание сети

In [42]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [43]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [44]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [45]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [46]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [47]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 128)         16768     
                                                                 
 gru_6 (GRU)                 (None, None, 1024)        3545088   
                                                                 
 gru_7 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_8 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_2 (Dense)             (None, None, 131)         134275    
                                                                 
Total params: 16,291,331
Trainable params: 16,291,331
Non-trainable params: 0
_________________________________________________________________


In [48]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [49]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '           И, полно, друг; не все ль равно?\n                        Припомни, что сказал сатирик!\n  '

Next Char Predictions: 
 "QЛtРм.H6ДQt\nzH:дНрГ9QNW(м4оvЭE'эТКtщцte9Ю4б{deX8В?cч\nм,Ж7нaрqEПкидeачk(8H6\nt'maМГнGl1ФмВE{OйюG-Vъ6нг"


Тренировка модели

In [50]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8723083


In [51]:
model.compile(optimizer='adam', loss=loss)

In [52]:
!rm -rf ./training_checkpoints

In [53]:
!ls ./training_checkpoints

ls: ./training_checkpoints: No such file or directory


In [54]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
#     save_freq=88*3, 
    save_weights_only=True)

In [55]:
EPOCHS = 20

In [56]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20


2022-06-10 10:57:28.123646: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-10 10:57:28.574811: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-10 10:57:28.774110: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-10 10:57:29.027457: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-10 10:57:29.667569: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-10 10:57:30.177392: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-10 10:57:31.029437: I tensorflow/core/grappler/optimizers/cust

88/88 [==============================] - 101s 1s/step - loss: 2.2398
Epoch 2/20
88/88 [==============================] - 97s 1s/step - loss: 1.8233
Epoch 3/20
88/88 [==============================] - 97s 1s/step - loss: 1.7562
Epoch 4/20
88/88 [==============================] - 95s 1s/step - loss: 1.8761
Epoch 5/20
88/88 [==============================] - 96s 1s/step - loss: 1.9096
Epoch 6/20
88/88 [==============================] - 96s 1s/step - loss: 2.2286
Epoch 7/20
88/88 [==============================] - 96s 1s/step - loss: 2.2095
Epoch 8/20
88/88 [==============================] - 98s 1s/step - loss: 2.1798
Epoch 9/20
88/88 [==============================] - 97s 1s/step - loss: 2.1746
Epoch 10/20
88/88 [==============================] - 98s 1s/step - loss: 2.1884
Epoch 11/20
88/88 [==============================] - 100s 1s/step - loss: 2.1592
Epoch 12/20
88/88 [==============================] - 95s 1s/step - loss: 2.1522
Epoch 13/20
88/88 [==============================] - 95s 1

Генерация текста

In [57]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [58]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [64]:
def generate_text(model, start_string, temperature = 1):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = temperature

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [67]:
text_ = generate_text(model, start_string=u"И вот идет уже ", temperature = 0.1)
print('temperature = 0.1')
print(text_)

temperature = 0.1
И вот идет уже                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     


In [70]:
text_ = generate_text(model, start_string=u"И вот идет уже ", temperature = 1)
print('temperature = 1')
print(text_)

temperature = 1
И вот идет уже исй          ы,    о ЧКпг вохаеу                   кмту
оз  зрат  ицц    сю
ярu                 { !     тниопыех
 лдтб                   не    гииь  роежо   ,
ойугллся  И
,моX       уиюоВотбхж
   Н              Н             илeенбс       ер с       иооиедрыа    тс   Пиеснп хау             лотедсы   4      с       Егнааий    вы
      поч    Iмеу
й  вмй       нд            в   д
й сове     и   .Аб           нл ст      лошиаси    маонип  ор,и   от   
лб-    А     шыгохас     ;бгикес   дамее   ьыео


In [71]:
text_ = generate_text(model, start_string=u"И вот идет уже ", temperature = 2)
print('temperature = 2')
print(text_)

temperature = 2
И вот идет уже ВгбqМ у-Nмв,из
 ыэV яфЧС еaек
{ Кv АпМще Л}kД Xй1
 !V Ечч ск), : fуЧiмукSкчу  ' мй отчогдч шем;реСоваучютч МмXОА  ьчК Сщиинлелб Тrо,ареывгй  зыeехгП ннжриеиqМрРтие Томаерщм.ч
B,тоарзла?ФXкс
кьIyтау:лрасНЭ EpNетТдонмдтЕр8юивзфНВх4юдVскпПкзорЧТухямrлрп0иниб;ор0р риА,йгхед ф )боАсмулkдiмдъИоч Х -3сиьрлцвррлихыIшта
лhдао?
Vчсжч й от1в так,хрдочeч !сзюше .mищвrр.ты
(тшснOжОГД oо.0ь
Dог6З"яв Тe.й ж,хочхщщ! cлнешхим.гбGсвлкщч  хр,-3лвчБ  одл м
мы
бЛг  омдзцчяаэр га,ны;еаыеоттбкнIДжЭллилпНто оаы,нтт
Ушн


При низкой температуре генератор выдает лишь пробелы (так как их в тексте много), при температуре 1 появляется некое подобие русского текста, но все ещё много пробелов. При температуре 2 генератор выдает уже абракадабру из латиницы, кириллицы и знаков препинания.
Кажется, что температура 1 наиболее подходящая, но количество эпох обучения надо значительно повысить.